# Existing Datasets

This notebook provides two ways of generating expert demostration. 

We have 5301 data provided by the VirtualHome environment. The data can be found with the link: http://virtual-home.org/release/watch_and_help/watch_data.zip

The data contains train and test split, under ```gather_data_actiongraph_test.json``` and ```gather_data_actiongraph_train.json``` respectively. 

You can get a list of data using the key ```data['train_data'] #(or 'test_data')```, and a list of data will be returned. There are several important keys you can access by calling ```data['train_data'][i].keys()```:

- name: name of this epsiode 
- valid_action_with_walk: actions taken to achieve goal
- init_graph: starting condition of the episode
- graphs: history of graphs that the episode produced 
- goal: the goal states of this episode
- task_name: category of the task
- env_id: id of the environment used for this episode


In [ ]:
import json

path = "Path/to/gather_data_actiongraph_{train/test/new_test}.json"
f = open(path)
data = json.load(f)
print(data.keys()) # dict_keys(['train_data']) / dict_keys(['test_data']) / dict_keys(['new_test_data'])
print(len(data['train_data'])) # 5301
print('Task type: {}'.format(data['train_data'][0]['task_name'])) # read_book
print('Goals to achieve: {}'.format(data['train_data'][0]['goal'])) # Goals to achieve: ['on_cupcake_coffeetable', 'on_pudding_coffeetable', 'on_apple_coffeetable', 'on_apple_coffeetable', 'holds_book', 'sit_sofa']
print(data['train_data'][0]['valid_action_with_walk']) # print out actions


# Our Approach to create more data

In order to create more data for training, we need to generate more data in order to create customized dataset for our need. Our solution to do this is to leverage existing planner discussed in [Language Models as Zero-Shot Planners](https://arxiv.org/abs/2201.07207). The following code demonstrate our modification on the existing planner to generate more VirtualHome-executable data. 

The algorithm to generate data can be found in the paper. In addition to the existing algorithm where only one prompt is fed into the model for the prediction of a sequence of actions, we further increase the number of it until one valid epsiode can be generated for a given task. We also define parsers to convert the templated sentence output into structured script which can be executed by VirtualHome.

## Some Code Reused From the Paper Repo

### Setup

In [1]:
!git clone https://github.com/huangwl18/language-planner
%cd language-planner
!pip install -r requirements.txt
%cd src

Cloning into 'language-planner'...
remote: Enumerating objects: 69, done.
remote: Counting objects: 100% (69/69), done.
remote: Compressing objects: 100% (57/57), done.
remote: Total 69 (delta 10), reused 66 (delta 10), pack-reused 0
Unpacking objects: 100% (69/69), done.
/content/language-planner
     |████████████████████████████████| 3.4 MB 4.3 MB/s 
     |████████████████████████████████| 14.8 MB 28.4 MB/s 
     |████████████████████████████████| 85 kB 5.9 MB/s 
     |████████████████████████████████| 831.4 MB 2.8 kB/s 
     |████████████████████████████████| 147 kB 67.3 MB/s 
     |████████████████████████████████| 895 kB 52.1 MB/s 
     |████████████████████████████████| 596 kB 63.9 MB/s 
     |████████████████████████████████| 67 kB 6.8 MB/s 
     |████████████████████████████████| 3.3 MB 52.4 MB/s 
     |████████████████████████████████| 1.2 MB 53.7 MB/s 
     |████████████████████████████████| 21.0 MB 1.2 MB/s 
     |████████████████████████████████| 23.2 MB 1.2 MB/s 
     |██

/content/language-planner/src


Import packages and specify which GPU to be used

In [2]:
import openai
import numpy as np
import torch
from sentence_transformers import SentenceTransformer
from sentence_transformers import util as st_utils
import json

GPU = 0
if torch.cuda.is_available():
    torch.cuda.set_device(GPU)
OPENAI_KEY = None  # replace this with your OpenAI API key, if you choose to use OpenAI API

### Define hyperparemeters for plan generation

Select the source to be used (**OpenAI API** or **Huggingface Transformers**) and the two LMs to be used (**Planning LM** for plan generation, **Translation LM** for action/example matching). Then define generation hyperparameters.

Available language models for **Planning LM** can be found from:
- [Huggingface Transformers](https://huggingface.co/models?pipeline_tag=text-generation&sort=downloads)
- [OpenAI API](https://beta.openai.com/docs/engines) (you would need to paste your OpenAI API key from your account to `openai.api_key` below)

Available language models for **Translation LM** can be found from:
- [Sentence Transformers](https://huggingface.co/sentence-transformers)

In [3]:
source = 'huggingface'
planning_lm_id = 'gpt2-large'  # see comments above for all options
translation_lm_id = 'stsb-roberta-large'  # see comments above for all options
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if source == 'openai':
    openai.api_key = OPENAI_KEY
    sampling_params = \
            {
                "max_tokens": 10,
                "temperature": 0.6,
                "top_p": 0.9,
                "n": 10,
                "logprobs": 1,
                "presence_penalty": 0.5,
                "frequency_penalty": 0.3,
                "stop": '\n'
            }
elif source == 'huggingface':
    sampling_params = \
            {
                "max_tokens": 10,
                "temperature": 0.1,
                "top_p": 0.9,
                "num_return_sequences": 10,
                "repetition_penalty": 1.2,
                'use_cache': True,
                'output_scores': True,
                'return_dict_in_generate': True,
                'do_sample': True,
            }

### Planning LM Initialization
Initialize **Planning LM** from either **OpenAI API** or **Huggingface Transformers**. Abstract away the underlying source by creating a generator function with a common interface.

In [4]:
def lm_engine(source, planning_lm_id, device):
    if source == 'huggingface':
        from transformers import AutoModelForCausalLM, AutoTokenizer
        tokenizer = AutoTokenizer.from_pretrained(planning_lm_id)
        model = AutoModelForCausalLM.from_pretrained(planning_lm_id, pad_token_id=tokenizer.eos_token_id).to(device)

    def _generate(prompt, sampling_params):
        if source == 'openai':
            response = openai.Completion.create(engine=planning_lm_id, prompt=prompt, **sampling_params)
            generated_samples = [response['choices'][i]['text'] for i in range(sampling_params['n'])]
            # calculate mean log prob across tokens
            mean_log_probs = [np.mean(response['choices'][i]['logprobs']['token_logprobs']) for i in range(sampling_params['n'])]
        elif source == 'huggingface':
            input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)
            prompt_len = input_ids.shape[-1]
            output_dict = model.generate(input_ids, max_length=prompt_len + sampling_params['max_tokens'], **sampling_params)
            # discard the prompt (only take the generated text)
            generated_samples = tokenizer.batch_decode(output_dict.sequences[:, prompt_len:])
            # calculate per-token logprob
            vocab_log_probs = torch.stack(output_dict.scores, dim=1).log_softmax(-1)  # [n, length, vocab_size]
            token_log_probs = torch.gather(vocab_log_probs, 2, output_dict.sequences[:, prompt_len:, None]).squeeze(-1).tolist()  # [n, length]
            # truncate each sample if it contains '\n' (the current step is finished)
            # e.g. 'open fridge\n<|endoftext|>' -> 'open fridge'
            for i, sample in enumerate(generated_samples):
                stop_idx = sample.index('\n') if '\n' in sample else None
                generated_samples[i] = sample[:stop_idx]
                token_log_probs[i] = token_log_probs[i][:stop_idx]
            # calculate mean log prob across tokens
            mean_log_probs = [np.mean(token_log_probs[i]) for i in range(sampling_params['num_return_sequences'])]
        generated_samples = [sample.strip().lower() for sample in generated_samples]
        return generated_samples, mean_log_probs

    return _generate

generator = lm_engine(source, planning_lm_id, device)

Downloading:   0%|          | 0.00/666 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.02G [00:00<?, ?B/s]

### Translation LM Initialization
Initialize **Translation LM** and create embeddings for all available actions (for action translation) and task names of all available examples (for finding relevant example)

In [5]:
# initialize Translation LM
translation_lm = SentenceTransformer(translation_lm_id).to(device)

# create action embeddings using Translated LM
with open('available_actions.json', 'r') as f:
    action_list = json.load(f)

action_list_embedding = translation_lm.encode(action_list, batch_size=512, convert_to_tensor=True, device=device)  # lower batch_size if limited by GPU memory

# create example task embeddings using Translated LM
with open('available_examples.json', 'r') as f:
    available_examples = json.load(f)

example_task_list = [example.split('\n')[0] for example in available_examples]  # first line contains the task name
example_task_embedding = translation_lm.encode(example_task_list, batch_size=512, convert_to_tensor=True, device=device)  # lower batch_size if limited by GPU memory


Downloading:   0%|          | 0.00/748 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.92k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/674 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/191 [00:00<?, ?B/s]

## Define Functions For our Purpose

As illustrated in [Language Models are Few-Shot Learners](https://arxiv.org/abs/2005.14165), we are predicting the next action with prompts. To further achieve better performance comparing to the method illustrated in 'Language Models as Zero-Shot Planners', we will try different numbers of k as our way to feed LM a prompt.



### Print out Human labelled episode examples



In [6]:
print("Number of expert demostrations: {}".format(len(available_examples)))

for i in range(5):
    print(available_examples[i])
    print("====================")

Number of expert demostrations: 5088
Task: Write an email
Step 1: Walk to home office
Step 2: Walk to computer
Step 3: Find computer
Step 4: Turn to computer
Step 5: Look at computer
Step 6: Walk to computer
Step 7: Find chair
Step 8: Sit on chair
Step 9: Find keyboard
Step 10: Grab keyboard
Step 11: Find mouse
Step 12: Grab mouse
Step 13: Type on keyboard
Task: Take shower
Step 1: Find clothes dress
Step 2: Find towel
Step 3: Walk to bathroom
Step 4: Walk to shower
Step 5: Find shower
Task: Watch TV
Step 1: Find remote control
Step 2: Grab remote control
Step 3: Find television
Step 4: Switch on television
Step 5: Turn to television
Step 6: Watch television
Step 7: Switch off television
Step 8: Put back remote control
Task: Hang up jacket
Step 1: Walk to home office
Step 2: Walk to clothes jacket
Step 3: Find clothes jacket
Step 4: Grab clothes jacket
Step 5: Walk to wall
Step 6: Find hanger
Step 7: Put clothes jacket on hanger
Task: Wash clothes
Step 1: Walk to bathroom
Step 2: Walk 

### Now we use some of these labelled examples as prompt to generate plans

In [76]:
# helper function for finding similar sentence in a corpus given a query
# gives random value when there is a tie in similarity scores 
# skip_tie argument defines if we should skip values of the same cos score
def find_k_most_similar(query_str, corpus_embedding, k = 1, skip_tie = False):
    query_embedding = translation_lm.encode(query_str, convert_to_tensor=True, device=device)
    # calculate cosine similarity against each candidate sentence in the corpus
    cos_scores = st_utils.pytorch_cos_sim(query_embedding, corpus_embedding)[0].detach().cpu().numpy()
    unique_vals = np.unique(np.sort(cos_scores))[::-1]
    if k > unique_vals.shape[0]:
        print("[WARNING] There are not enough distinct similarity scores for K = {}. Getting K={} instead".format(k, unique_vals.shape[0]))
        k = unique_vals.shape[0]
    order = 0
    inds = np.array([]).astype(np.int32)
    while inds.shape[0] < k:
        num_selected = 1 if skip_tie else k-inds.shape[0] 
        selected_ind = np.random.choice(np.flatnonzero(cos_scores == unique_vals[order]), size = (num_selected))
        inds = np.concatenate((inds, selected_ind), axis = 0)
        order += 1
    scores = cos_scores[inds]
    return inds, scores


In [144]:
import collections

def generate(task, k, task_embedding, character='char0', condition=None, print_out=False):
    #mapping from mapped actions to actions that we can process in our LM 
    def __verb_mapping(verb):
        mapping = {'find':'grab'}
        return mapping[verb] if verb in mapping else verb

    #implement few shot conditional prompted LM to generate actions  
    inds, _ = find_k_most_similar(task, example_task_embedding, k = k)
    curr_prompt = ''
    for example_idx in inds:
        example = available_examples[example_idx]
        # print example and query task
        if print_out:
            print('-'*10 + ' GIVEN EXAMPLE ' + '-'*10)
            print(example)
            print('-'*10 + ' EXAMPLE END ' + '-'*10)
        curr_prompt += f'{example}\n\n'
    curr_prompt += f'Task: {task}\n'
    if condition: curr_prompt += f'Condition: {condition}'
    if print_out: print(f'\nTask: {task}')
    actions = []
    
    # Alter code to support few shot prompting
    for step in range(1, MAX_STEPS + 1):
        best_overall_score = -np.inf
        # query Planning LM for single-step action candidates
        samples, log_probs = generator(curr_prompt + f'\nStep {step}:', sampling_params)
        for sample, log_prob in zip(samples, log_probs):
            idxs, scores = find_k_most_similar(sample, action_list_embedding)
            most_similar_idx, matching_score = idxs[0], scores[0]
            # rank each sample by its similarity score and likelihood score
            overall_score = matching_score + BETA * log_prob
            translated_action = action_list[most_similar_idx]
            # heuristic for penalizing generating the same action as the last action
            if step > 1 and translated_action == previous_action:
                overall_score -= 0.5
            # find the translated action with highest overall score
            if overall_score > best_overall_score:
                best_overall_score = overall_score
                best_action = translated_action

        # terminate early when either the following is true:
        # 1. top P*100% of samples are all 0-length (ranked by log prob)
        # 2. overall score is below CUTOFF_THRESHOLD
        # else: autoregressive generation based on previously translated action
        top_samples_ids = np.argsort(log_probs)[-int(P * len(samples)):]
        are_zero_length = all([len(samples[i]) == 0 for i in top_samples_ids])
        below_threshold = best_overall_score < CUTOFF_THRESHOLD
        if are_zero_length:
            if print_out: print(f'\n[Terminating early because top {P*100}% of samples are all 0-length]')
            break
        elif below_threshold:
            if print_out: print(f'\n[Terminating early because best overall score is lower than CUTOFF_THRESHOLD ({best_overall_score} < {CUTOFF_THRESHOLD})]')
            break
        else:
            info = best_action.split(' ')
            if len(info) == 2: # e.g. find shower
                 actions += ['<{}> [{}] <{}>'.format(character, __verb_mapping(info[0]), info[1])]
            elif len(info) == 3: # e.g. go to Bathroom
                 actions += ['<{}> [{}] <{}>'.format(character, __verb_mapping(info[0]), info[2])]
            elif len(info) == 4: # e.g. pour facecream into pajamas
                actions += ['<{}> [{}] <{}> <{}>'.format(character, __verb_mapping(info[0]+info[2]), info[1], info[3])]
            previous_action = best_action
            formatted_action = (best_action[0].upper() + best_action[1:]).replace('_', ' ') # 'open_fridge' -> 'Open fridge'
            curr_prompt += f'\nStep {step}: {formatted_action}'
            if print_out: print(f'Step {step}: {formatted_action}')
    
    return actions


In [121]:
# Define parameters
MAX_STEPS = 10  # maximum number of steps to be generated
CUTOFF_THRESHOLD = 0.8 # early stopping threshold based on matching score and likelihood score
P = 0.7  # hyperparameter for early stopping heuristic to detect whether Planning LM believes the plan is finished
BETA = 0.3  # weighting coefficient used to rank generated samples

In [145]:
action_script = generate(task='Drink beer', k = 3, task_embedding=example_task_embedding, print_out=True)
print(action_script)

---------- GIVEN EXAMPLE ----------
Task: Drink
Step 1: Walk to dining room
Step 2: Walk to cup
Step 3: Find cup
Step 4: Grab cup
Step 5: Walk to sink
Step 6: Walk to water
Step 7: Find water
Step 8: Grab water
Step 9: Pour water into cup
Step 10: Drink cup
---------- EXAMPLE END ----------
---------- GIVEN EXAMPLE ----------
Task: Drink
Step 1: Walk to dining room
Step 2: Walk to cupboard
Step 3: Find cupboard
Step 4: Open cupboard
Step 5: Find water glass
Step 6: Grab water glass
Step 7: Walk to water
Step 8: Find water
Step 9: Grab water
Step 10: Pour water into water glass
Step 11: Drink water glass
Step 12: Rinse water glass
Step 13: Put water glass on cupboard
---------- EXAMPLE END ----------
---------- GIVEN EXAMPLE ----------
Task: Drink
Step 1: Walk to dining room
Step 2: Walk to freezer
Step 3: Find freezer
Step 4: Open freezer
Step 5: Find drinking glass
Step 6: Grab drinking glass
Step 7: Find water
Step 8: Drink drinking glass
---------- EXAMPLE END ----------

Task: Drin

# Connect Virtual Home Backend with code

## Set up using this [github](https://github.com/xavierpuigf/virtualhome/blob/master/demo/unity_demo.ipynb)

In [81]:
import os
%cd /content
if 'google.colab' in str(get_ipython()):
    print('Running on CoLab')
    osname = "linux"
    !pip install git+https://github.com/xavierpuigf/colabstreamer
    import colabstreamer
    colabstreamer.config_all()
    _xorg = colabstreamer.open_xorg()
    # Clone VirtualHome
    !git clone https://github.com/xavierpuigf/virtualhome.git
    %cd /content/virtualhome
    !pip install -r requirements.txt
else:
    %cd ..
    from sys import platform
    if platform == "darwin":
        osname = "macos"
    elif platform == "linux":
        osname = "linux"
    elif platform == "windows" or platform == 'win32':
        osname = "windows"

/content
Running on CoLab
  Cloning https://github.com/xavierpuigf/colabstreamer to /tmp/pip-req-build-jzyj5ouq
  Running command git clone -q https://github.com/xavierpuigf/colabstreamer /tmp/pip-req-build-jzyj5ouq
     |████████████████████████████████| 179 kB 7.5 MB/s 
  Created wheel for colabstreamer.py: filename=colabstreamer.py-0.1-py3-none-any.whl size=3830 sha256=45a6dadc9d4cdafeb410d79c60dade273940e07e9c86e892c2a586a5849dcde2
  Stored in directory: /tmp/pip-ephem-wheel-cache-7lk_rhoz/wheels/7e/df/62/68e8e5813ab1ac489ca123bc4051459c61ba909df0ec7ac404
Successfully built colabstreamer.py
Will Install xvfb
Will Install xserver-xorg
Will Install mesa-utils
Will Install xinit
Will Install xdotool
Will Install linux-generic
Will Install xterm
Will Install htop
Will Install i3
Will Install xloadimage
Will Install libgtk2.0-0
Will Install libgconf-2-4
Cloning into 'virtualhome'...
remote: Enumerating objects: 27196, done.
remote: Counting objects: 100% (580/580), done.
remote: Compres

In [93]:
if not os.path.isfile(f"{osname}_exec.zip"):
    ! wget http://virtual-home.org/release/simulator/v2.0/v2.3.0/linux_exec.zip
    ! unzip -q linux_exec.zip

--2022-04-03 03:53:29--  http://virtual-home.org/release/simulator/v2.0/v2.3.0/linux_exec.zip
Resolving virtual-home.org (virtual-home.org)... 128.30.100.223
Connecting to virtual-home.org (virtual-home.org)|128.30.100.223|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 287682964 (274M) [application/zip]
Saving to: ‘linux_exec.zip’

linux_exec.zip      100%[===================>] 274.36M  9.04MB/s    in 31s     

2022-04-03 03:54:00 (8.90 MB/s) - ‘linux_exec.zip’ saved [287682964/287682964]



In [125]:
%matplotlib notebook
%cd demo
import IPython.display
import glob
from utils_demo import *
from sys import platform
import sys
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm
sys.path.append('../simulation')
from unity_simulator.comm_unity import UnityCommunication
from unity_simulator import utils_viz

[Errno 2] No such file or directory: 'demo'
/content/virtualhome/demo


In [ ]:
mode = 'auto' # auto / manual
if mode == 'auto':
    if platform == 'darwin':
        exec_file = '../macos_exec*'
    else:
        exec_file = '../linux_exec*.x86_64'
    file_names = glob.glob(exec_file)
    if len(file_names) > 0:
        file_name = file_names[0]
        comm = UnityCommunication(file_name=file_name, port="8082", x_display="0")
    else:
        print("Error: executable path not found.")
else:
    comm = UnityCommunication()

## Add an main agent and generate actions

In [ ]:
#reset one environment 
comm.reset(0)
comm.add_character('chars/Female2', initial_room='kitchen')
s, g = comm.environment_graph()

In [141]:
# Define parameters
MAX_STEPS = 10  # maximum number of steps to be generated
CUTOFF_THRESHOLD = 0.8 # early stopping threshold based on matching score and likelihood score
P = 0.8  # hyperparameter for early stopping heuristic to detect whether Planning LM believes the plan is finished
BETA = 0.3  # weighting coefficient used to rank generated samples


---------- GIVEN EXAMPLE ----------
Task: Drink
Step 1: Walk to dining room
Step 2: Walk to cupboard
Step 3: Find cupboard
Step 4: Open cupboard
Step 5: Find cup
Step 6: Grab cup
Step 7: Walk to sink
Step 8: Find faucet
Step 9: Turn to faucet
Step 10: Put cup on sink
Step 11: Switch on faucet
Step 12: Find water
Step 13: Grab cup
Step 14: Put cup on sink
Step 15: Grab cup
Step 16: Switch off faucet
Step 17: Drink cup
---------- EXAMPLE END ----------
---------- GIVEN EXAMPLE ----------
Task: Drink
Step 1: Walk to dining room
Step 2: Walk to water
Step 3: Find water
Step 4: Grab water
Step 5: Find drinking glass
Step 6: Pour water into drinking glass
Step 7: Find drinking glass
Step 8: Grab drinking glass
Step 9: Drink drinking glass
---------- EXAMPLE END ----------

Task: Drink beer
Step 1: Walk to dining room
Step 2: Walk to water
Step 3: Find water

[Terminating early because best overall score is lower than CUTOFF_THRESHOLD (0.781432569026947 < 0.8)]


### Check if the data can be a valid sequence of action (debugging needed)

In [ ]:
success = False
attempt = 1
while not success or attempt <= 1000:
    data = generate(task='Drink beer', k = 2, task_embedding=example_task_embedding)
    success, message = comm.render_script(script=data,
                                      processing_time_limit=60,
                                      find_solution=False,
                                      image_width=320,
                                      image_height=240,  
                                      skip_animation=False,
                                      recording=True,
                                      save_pose_data=True,
                                      file_name_prefix='relax')
    attempt += 1
    print('Creating Trail {} out of 1000...{}'.format(attempt, success))
    
if not success:
    print('Data generated are all invalid in accordance to the given environment. Actions dumped.')

#### These functions are used for setting up environments

In [104]:
'''
def create_obj_json(class_name, category, id, properties=[], states=[]):
    return {'class_name': class_name, 
            'category': category, 
            'id': id, 
            'properties': properties, 
            'states': states}
def add_single_target(graph, item_json, location_node, proposition):
    add_node(graph, item_json)
    add_edge(graph, item_json['id'], proposition, location_node['id'])

# Place a task item into the environment
cat_item = create_obj_json('beer', 'Drinks', 1000)
sofa = find_nodes(graph, class_name='sofa')[0]
add_single_target(graph=graph, item_json=cat_item, location_node=sofa, proposition='ON')
success, message = comm.expand_scene(graph)
'''

#Code below not working

### Let's generate a batch of them (Too slow with the given pipeline)

In [ ]:
# Batch Generation

In [ ]:
data = []
tasks = ['Make Breakfast', 'Take Shower', 'Eat Apple', 'Reply Email']
num_gen_per_task = 10
for task in tasks:
    for n in num_gen_per_task:
        data += [generate(task='Take Shower', k = 3, task_embedding=example_task_embedding)]

In [ ]:
# write generated data into output file

fout = open('./generated_results.txt', 'w')


['[walk <bathroom>]', '[walk <shower>]', '[grab <shampoo>]', '[wash <hair>]']